In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [6]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print(f"shape of X [N,C,H,W]:{X.shape}")
    print(f"shape of y:{y.shape}{y.dtype}")
    break

shape of X [N,C,H,W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64


In [7]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"using {device}")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

using cpu
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [9]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0 :
            loss,current = loss.item(),(batch + 1)*len(X)
            print(f"loss:{loss:>7f}[{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"test error \n accuracy: {(100*correct):>0.1f}%,avg loss:{test_loss:>8f}\n")

In [15]:
epochs = 5
for t in range(epochs):
    print(f"epoch {t+1}\n--------------")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("done")

epoch 1
--------------
loss:1.925911[   64/60000]
loss:1.901410[ 6464/60000]
loss:1.777547[12864/60000]
loss:1.830849[19264/60000]
loss:1.737725[25664/60000]
loss:1.664801[32064/60000]
loss:1.696640[38464/60000]
loss:1.595984[44864/60000]
loss:1.631875[51264/60000]
loss:1.521486[57664/60000]
test error 
 accuracy: 0.0%,avg loss:1.535224

epoch 2
--------------
loss:1.596748[   64/60000]
loss:1.562905[ 6464/60000]
loss:1.406267[12864/60000]
loss:1.488330[19264/60000]
loss:1.384832[25664/60000]
loss:1.356558[32064/60000]
loss:1.373402[38464/60000]
loss:1.302591[44864/60000]
loss:1.342675[51264/60000]
loss:1.235802[57664/60000]
test error 
 accuracy: 0.0%,avg loss:1.262106

epoch 3
--------------
loss:1.334639[   64/60000]
loss:1.318434[ 6464/60000]
loss:1.146039[12864/60000]
loss:1.260834[19264/60000]
loss:1.146767[25664/60000]
loss:1.152797[32064/60000]
loss:1.171548[38464/60000]
loss:1.120085[44864/60000]
loss:1.158056[51264/60000]
loss:1.071498[57664/60000]
test error 
 accuracy: 0.0%

In [16]:
torch.save(model.state_dict(),"model.pth")
print("saved pytorch model state to model.pth")

saved pytorch model state to model.pth


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth",weights_only=True))

<All keys matched successfully>

In [22]:
classes = [
    "T - shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y = test_data[0][0],test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted,actual = classes[pred[0].argmax(0)],classes[y]
    print(f'predicted:"{predicted}",actual:"{actual}"')

predicted:"Ankle boot",actual:"Ankle boot"
